# Task 6 - Simulating Displacements Per Atom (DPA)

Displacements per atom (DPA) is one measure of damage within materials exposed to neutron irradiation. Damage energy can be tallied in OpenMC with MT reaction number 444 and DPA can be estimated.

In the case of DPA a neutronics code alone can't fully calculate the value as material science techniques are needed to account for the material and recombination effects. For example, after a displacement there is a chance that the atom relocates to it's original lattice position (recombination) and different atoms require different amounts of energy to [displace](https://fispact.ukaea.uk/wiki/Output_interpretation#DPA_and_KERMA). The DPA tally from neutronics is therefore only an estimate of the DPA.

The MT 444 / damage energy tally is in units of eV per source particle. Therefore the result needs scaling by the source intensity (in neutrons per second), the irradiation duration (in seconds) and the number of atoms in the volume.

This first stage sets up the geometry and materials for the simulation.

In [ ]:
import openmc
import json


# MATERIALS

density_of_iron_in_g_per_cm3 = 7.75
my_material = openmc.Material(name='Iron')
my_material.set_density('g/cm3', density_of_iron_in_g_per_cm3)
my_material.add_element('Fe', 1.0, percent_type='wo')

mats = openmc.Materials([my_material])


# GEOMETRY

# surfaces
outer_surface = openmc.Sphere(r=610, boundary_type='vacuum')

# cells
vessel_region = -outer_surface
vessel_cell = openmc.Cell(region=vessel_region)
vessel_cell.fill = my_material

universe = openmc.Universe(cells=[vessel_cell])
geom = openmc.Geometry(universe)


# SIMULATION SETTINGS

# Instantiate a Settings object
sett = openmc.Settings()
batches = 10
sett.batches = batches
sett.inactive = 0
sett.particles = 10000
sett.run_mode = 'fixed source'

# Create a DT point source
source = openmc.Source()
source.space = openmc.stats.Point((0, 0, 0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14e6], [1])
sett.source = source

This sets up the damage energy tally using the MT number 444. A list of MT numbers including their reaction discription can be found [here](https://t2.lanl.gov/nis/endf/mts.html).

In [ ]:
tallies = openmc.Tallies()

# added a cell tally for DPA to the iron vessel cell
cell_filter = openmc.CellFilter(vessel_cell)
reaction_tally = openmc.Tally(name='DPA')
reaction_tally.filters = [cell_filter]
reaction_tally.scores = ['444']  # note use of 444 in string format
tallies.append(reaction_tally)

This runs the simulation.

In [ ]:
model = openmc.model.Model(geom, mats, sett, tallies)
!rm *.h5
sp_filename = model.run()

This extracts the simulation results and calculates the total number of displacements for all atoms. From here, DPA can be found.

In [ ]:
sp = openmc.StatePoint(sp_filename)

# access the tally
tally = sp.get_tally(name='DPA')

df = tally.get_pandas_dataframe()

damage_energy_in_ev = df['mean'].sum()


print('Damage energy deposited per source neutron = ', damage_energy_in_ev, 'eV\n')

print('Two times the threshold energy of 40eV is needed to displace an atom')
displacements_per_source_neutron = damage_energy_in_ev / (2*40)
print('Displacements per source neutron = ', displacements_per_source_neutron, '\n')

print('Assuming about 80% remains after 20% recombine to original lattice locations')
displacements_per_source_neutron_with_recombination = displacements_per_source_neutron*0.8
print('Displacements per source neutron after recombination = ', displacements_per_source_neutron_with_recombination, '\n')

fusion_power = 3e9  # units Watts
energy_per_fusion_reaction = 17.6e6  # units eV
eV_to_Joules = 1.60218e-19  # multiplication factor to convert eV to Joules
number_of_neutrons_per_second = fusion_power / (energy_per_fusion_reaction * eV_to_Joules)
print('Number of neutrons per second', number_of_neutrons_per_second, '\n')

number_of_neutrons_per_year = number_of_neutrons_per_second * 60 * 60 * 24 * 365.25
print('Number of neutrons per full power year ', number_of_neutrons_per_year)

displacements_for_all_atoms = number_of_neutrons_per_year * displacements_per_source_neutron_with_recombination
print('Displacements for all atoms in the volume ', displacements_for_all_atoms, '\n')

print('Now the number of atoms in the volume must be found to find displacements per atom (DPA)')

This section carries out a volume calculation for the CSG cell. Particles are created randomly within the bounding box (of known volume). Upon creation of particles a check is carried out to see if the particle is also in the CSG volume. The ratio of the particles made in the bounding box to the particles made in the CSG cell is then used to determine the volume of the CSG cell. And error for the CSG cell is also determined.

In [ ]:
# volume calculates for materials require a bounding box
lower_left = (-1000, -1000, -1000.)
upper_right = (1000, 1000, 1000.)
material_vol_calc = openmc.VolumeCalculation([my_material], 100000, lower_left, upper_right)

cell_vol_calc = openmc.VolumeCalculation([vessel_cell], 100000)

settings = openmc.Settings()
settings.volume_calculations = [cell_vol_calc, material_vol_calc]
settings.run_mode = 'volume'
settings.export_to_xml()

!rm *.h5
openmc.run()

cell_vol_calc_results = openmc.VolumeCalculation.from_hdf5('volume_1.h5')
print('\nvessel_cell volume', cell_vol_calc_results.volumes[1], 'cm3')

material_vol_calc_results = openmc.VolumeCalculation.from_hdf5('volume_2.h5')
print('my_material volume', material_vol_calc_results.volumes[1], 'cm3')


# the cell_vol_calc_results are combined with errors, you can access the
# result on its own using the .nominal_value method

volume_of_firstwall_cell = cell_vol_calc_results.volumes[1].nominal_value

iron_atomic_mass_in_g = 55.845*1.66054E-24  # molar mass multiplier by the atomic mass unit (u)
number_of_iron_atoms = volume_of_firstwall_cell * density_of_iron_in_g_per_cm3 / (iron_atomic_mass_in_g)

print('Number of iron atoms in the firstwall ', number_of_iron_atoms)

Therefore, as the total number of atoms and the total number of displacements is known, DPA can be found.

In [ ]:
DPA = displacements_for_all_atoms / number_of_iron_atoms

print('DPA =', DPA)

**Learning Outcomes for Task 6:**

- Damage energy deposited can be found with the OpenMC MT 444 tally.
- The volume of a cell can be found using the stochastic volume method.
- Post tally calculations are sometimes required to convert neutronics numbers into something more useful.